In [2]:
# imports
from scipy.io import loadmat
import numpy as np
import pandas as pd
from tsfresh import extract_features
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters

In [3]:
# load data
matfile_baseline = loadmat('data/raw/Ach-AT/00_0315_ach-at_0.mat')
matfile_ach = loadmat('data/raw/Ach-AT/00_0315_ach-at_1.mat')
matfile_at = loadmat('data/raw/Ach-AT/00_0315_ach-at_2.mat')

In [6]:
def make_windows(matfile, window_size = 6000):
    # retrieve the relevant data from the dictionaries
    df = pd.DataFrame(matfile['filt_data'].copy())
    df = df.T
    time = pd.DataFrame(matfile['filt_t'].copy())
    time = time.T

    # set up the time column
    time.columns = ['t']
    time['t'] = time['t'] - 60000 # make time start at 0
    time['window_id'] = time['t'] / window_size # n second windows
    time['window_id'] = time['window_id'].astype(int)

    # join the electrode signals with time
    df = pd.concat([time, df], axis=1)

    return df

In [6]:
matfile_baseline['config']

array([[ 0, 24, 26, 29, 32, 35, 37,  0],
       [21, 22, 25, 30, 31, 36, 39, 40],
       [19, 20, 23, 28, 33, 38, 41, 42],
       [16, 17, 18, 27, 34, 43, 44, 45],
       [15, 14, 13,  4, 57, 48, 47, 46],
       [12, 11,  8,  3, 58, 53, 50, 49],
       [10,  9,  6,  1, 60, 55, 52, 51],
       [ 0,  7,  5,  2, 59, 56, 54,  0]], dtype=uint8)

In [7]:
df_base = make_windows(matfile_baseline)
df_ach = make_windows(matfile_ach)
df_at = make_windows(matfile_at)

In [10]:
cols = ['t','window_id',4,27,34,57]
df_base[cols]

,t,window_id,4,27,34,57
0,1.000000,0,-124.371209,-53.851959,-74.495868,-137.695972
1,2.000000,0,-124.485424,-53.731875,-74.128554,-137.754658
2,3.000000,0,-124.598914,-53.611991,-73.763533,-137.812580
3,4.000000,0,-124.711680,-53.492304,-73.400813,-137.869738
4,5.000000,0,-124.823721,-53.372811,-73.040403,-137.926133
...,...,...,...,...,...,...
179995,179996.000001,29,-21.193228,-85.037623,-55.347268,-25.560794
179996,179997.000001,29,-21.352978,-85.083660,-55.397883,-25.730796
179997,179998.000001,29,-21.512294,-85.128809,-55.448605,-25.900408
179998,179999.000001,29,-21.671170,-85.173068,-55.499432,-26.069626


In [8]:
# train on four electrodes - choose 4,27,34 and 57 (the centre-most ones)
df_base_4 = df_base[['t','window_id',4,27,34,57]]
df_base_4.loc[:,'y'] = 0
df_ach_4 = df_ach[['t','window_id',4,27,34,57]]
df_ach_4.loc[:,'y'] = 1
df_at_4 = df_at[['t','window_id',4,27,34,57]]
df_at_4.loc[:,'y'] = 2
# this could be done more efficiently, build into the function


df_4 = pd.concat([df_base_4, df_ach_4, df_at_4], axis=0, ignore_index=True)
y_4 = df_4[['window_id','y']]
df_4 = df_4.drop(columns = ['y'])
y_4.to_hdf('0_0315_4_electrodes_y.h5', key = 'y', complevel = 9)

/home/kartikey/anaconda3/envs/ts/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/kartikey/anaconda3/envs/ts/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
y_4

,window_id,y
_temporary_index_column,,
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
539995,149,2
539996,149,2
539997,149,2


In [9]:
# extract minimal features and see how long it takes
X_minimal = extract_features(df_4, column_id='window_id', column_sort='t', default_fc_parameters=MinimalFCParameters())

Feature Extraction: 100%|██████████| 10/10 [00:00<00:00, 39.56it/s]


In [17]:
# extract more features
X_eff = extract_features(df_4, column_id='window_id', column_sort='t', default_fc_parameters=EfficientFCParameters())

Feature Extraction: 100%|██████████| 10/10 [11:36<00:00, 69.67s/it]


In [10]:
X_minimal.to_hdf('0_0315_4_electrodes_min.h5', key = 'features', complevel = 9)

In [19]:
X_eff.to_hdf('0_0315_4_electrodes_eff.h5',key = 'features', complevel = 9)